In [1]:
import pandas as pd 
import numpy as np
import sys
sys.path.append("../utils/")
from recsys_metric import mrr

time: 173 ms (started: 2022-06-14 03:32:31 +00:00)


In [ ]:
itemcf = pd.read_csv("../datasets/results/leader_itemcf_k8000_train_purchases_5m_leader_candidate_filter_0613.csv")
itemcf.head()

In [ ]:
tfidf = pd.read_csv("../datasets/results/leader_tfidf_train_purchases_con_test_last_05_0601.csv")
tfidf.head()

In [3]:
# def ensable_df(df1,df2,ratio=0.5):
#     '''
#     given two df of rank return weighted rank
#     ratio = df1_ration:df2_ration = 1:0.5 (default)
#     '''
#     df1_r,df2_r = 1,1*ratio
#     df1["rank"] ,df2["rank"]  = 1/df1["rank"]*df1_r,1/df2["rank"]*df2_r
#     re_lst =[]
#     for idx in range(100,len(df1)+1,100):
#         buf_lst=[]
#         df1_s = df1.iloc[idx-100:idx]
#         df2_s = df2.iloc[idx-100:idx]
#         s_id = df1_s.session_id.unique()
#         total_df = (pd.concat([df1_s,df2_s],ignore_index=True))
#         for item_id in total_df.item_id.unique():
#             if item_id != -1:
#                 buf_lst.append((s_id[0],item_id,sum(total_df["rank"][total_df["item_id"]==item_id])))
#         buf_lst.sort(key=lambda x: x[2],reverse=True)
#         re_lst+=(buf_lst[:100])#take top 100
#     df = pd.DataFrame(re_lst,columns=['session_id', 'item_id', 'rank'])
#     return df


def ensable_df(itemcf_df, tfidf_df,ratio=0.5): 
    tmp = pd.merge(itemcf_df, tfidf_df, on=["session_id", "item_id"], how="outer", suffixes=["_itemcf", "_tfidf"]).fillna(10000)
    tmp['rank_itemcf_rec'] = 1/tmp['rank_itemcf']
    tmp['rank_tfidf_rec'] = 1/tmp['rank_tfidf']
    tmp.loc[tmp['rank_itemcf_rec']<0.01,'rank_itemcf_rec']=0
    tmp.loc[tmp['rank_tfidf_rec']<0.01,'rank_tfidf_rec']=0
    tmp['score'] = tmp['rank_itemcf_rec'] + ratio*tmp['rank_tfidf_rec']
    tmp = tmp.sort_values(['score'], ascending=False).groupby('session_id').head(100).sort_values(['session_id', 'score'], ascending=[True, False])
    tmp['rank'] = list(range(1,101))*tmp['session_id'].nunique()
    res_df = tmp.loc[:, ['session_id', 'item_id', 'rank']]
    return res_df


time: 1.31 ms (started: 2022-06-14 03:32:39 +00:00)


In [ ]:
itemcf = pd.read_csv("../datasets/results/itemcf_1m_en.csv")
tfidf = pd.read_csv("../datasets/results/tfidf_1m_en.csv")
ens_df = ensable_df(itemcf,tfidf,ratio=0.02)
eva_res = ens_df 
eva_label = pd.read_csv('../datasets/purchases_last_1m.csv')
mrr_score = mrr(eva_res, eva_label, overview=False)

print(mrr_score)

In [4]:
itemcf = pd.read_csv("../datasets/results/leader_itemcf_k8000_train_purchases_5m_leader_candidate_filter_0613.csv")
tfidf = pd.read_csv("../datasets/results/leader_tfidf_train_purchases_con_test_last_05_0601.csv")
res_df = ensable_df(itemcf,tfidf,ratio=0.01)
res_df.to_csv("../datasets/results/leader_en001_tfidf_train_purchases_con_test_last_05_itemcf_k8000_train_purchases_5m_leader_candidate_filter.csv", index=False)

time: 10.3 s (started: 2022-06-14 03:32:41 +00:00)


In [ ]:
# def multi_proc(df1_path,df2_path,n_jobs):
#     files = list(os.listdir(session_path))
# 	#split = int(len(files)/n_jobs)
#     count=0
#     jobs =[]
#     manager = mp.Manager()#define shared class
#     return_dict = manager.list()
#     while(count != len(files)):
#         for i in range(n_jobs):
#             if count == len(files):
#                 break
#             p = mp.Process(target=sessions2vec,args=(item_path,session_path+files[count],return_dict))
#             count+=1
#             jobs.append(p)
#             p.start()
#         for proc in jobs:
#             proc.join()
#     print(len(return_dict))
#     return return_dict.copy()